# Использование Python для работы с Outlook

*Текст лекции: Берников Д.А.

## Поиск и чтение писем

Для начала импортируем библиотеку win32 и другие необходимые библиотеки, которые были изучены в прошлых лекциях

In [1]:
import win32com.client
from win32com.client import Dispatch, constants
import pandas as pd
import numpy as np
import datetime #нужна для работы с датами

Создадим переменную, которая нужна для работы с Outlook. Для удобства назовем ее в честь приложения

In [2]:
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

Если мы попробуем вывести значение переменной, то получим текст, не несущий никакого значения. 

In [3]:
print(outlook)

Mapi


Применив команду `Folders`, мы сможем получить лист с почтовыми аккаунтами, которые есть в приложении, но, чтобы их увидеть, надо использовать функцию `Name`

In [4]:
accounts=outlook.Folders
accounts

<COMObject <unknown>>

In [5]:
for i in range(10): #выводит первые 10 аккаунтов
    try:
        box = accounts[i]
        name = box.Name
        print(i, name)
    except: #Нужно, чтобы код не выдавал ошибку, если аккаунтов меньше 10
        pass

0 Public Folders - Denis_Bernikov@mckinsey.com
1 CI Experience
2 My Outlook Data File(1)
3 Denis_Bernikov@mckinsey.com


Как видно, у меня 4 аккаунта, причем только 2 из них полноценные: CI Experience и Denis_Bernikov@mckinsey.com. Остальные аккаунты - вспомогательные.
Посмотрим, что находится в моем личном аккаунте :) Это можно сделать аналогично с помощью функции Folders

In [6]:
folders=accounts[3].Folders
#folders=accounts['Denis_Bernikov@mckinsey.com'].Folders 
#Выбор аккаунта можно сделать 2 способами: по номеру либо по названию
#Аналогично это будет работать  дальше с выбором папки
for i in range(200): #Папок у меня больше, чем 10, поэтому я сразу с запасом поставил 200
    try:
        box = folders[i]
        name = box.Name
        print(i, name)
    except:
        pass

0 Deleted Items
1 Inbox
2 Outbox
3 Sent Items
4 Read status
5 Reading
6 Reminders
7 Reminders by Igor and Sergey
8 RSS Subscriptions
9 Social Activity Notifications
10 Spam
11 Tasks
12 Templates
13 Quick Step Settings
14 the file so that changes to the file will be reflected in your item.
15 VA
16 Yammer Root
17 Contacts
18 Calendar
19 Conversation History
20 Drafts
21 Junk Email
22 Useful info
23 PersonMetadata
24 Sync Issues
25 Newsletter
26 Files
27 Archive
28 Auto replies
29 Birthdays
30 Box
31 Notes
32 Caring Leader
33 ExternalContacts
34 My Cloud
35 Green
36 Last day
37 Journal
38 Conversation Action Settings
39 Calendar responses


Посмотрим папку, письма из которой вы, итак, читали (надеюсь) – Newsletter

In [7]:
mails=folders['Newsletter'].Folders
for i in range(200):
    try:
        box = mails[i]
        name = box.SentOn
        print(i, name)
    except:
        pass

Мы вышли на уровень писем, и здесь уже не работает функция `Folders`. А вместо нее надо использовать `Items`. У сообщений нет атрибута `Name`, поэтому используем `Subject`

In [8]:
mails=folders['Newsletter'].Items
for i in range(200):
    try:
        box = mails[i]
        subject = box.Subject
        print(i, subject)
    except:
        pass

0 Experience Weekly: scoring in a basketball game for the client 
1 Experience Weekly: friendship and rescue operation in the mountains
2 Experience Weekly: secrets from the fashion industry
3 Experience Weekly: cross-country remote
4 Experience Weekly: guessing where your ED is from
5 Experience Weekly: Values, beard and world dominance
6 Experience Weekly: we are the champions
7 Experience Weekly: who shot Erik Fisher?
8 Experience Weekly: firstly, sport, and then wine
9 Experience Weekly: 600k from the charity auction 
10 Experience Weekly: nostalgia about sushi (on Wednesdays) po sredam 
11 Experience Weekly: where I was born
12 Experience Weekly: Who ya gonna call? Kuzbassters!
13 Experience Weekly: convincing with a knife
14 Experience Weekly: winners, winners – chicken dinners 
15 Experience Weekly: CEO with shisha, Thursday drinkers, and quarantine gifts
16 Experience Weekly: drums (again), pug-trainer, and toilet paper
17 Experience Weekly: drums, patriotism, yoga carpet misus

Складывается ощущение, что письма отсортированы не по дате. Чтобы в этом убедиться добавим вывод даты сообщения с помощью функции `SentOn`

In [9]:
for i in range(200):
    try:
        box = mails[i]
        subject = box.Subject
        date=box.SentOn.strftime("%d/%m/%Y") #вот и  пригодилась библиотека datetime
        print(i, date, subject)
    except:
        pass

0 31/07/2020 Experience Weekly: scoring in a basketball game for the client 
1 24/07/2020 Experience Weekly: friendship and rescue operation in the mountains
2 17/07/2020 Experience Weekly: secrets from the fashion industry
3 10/07/2020 Experience Weekly: cross-country remote
4 03/07/2020 Experience Weekly: guessing where your ED is from
5 26/06/2020 Experience Weekly: Values, beard and world dominance
6 19/06/2020 Experience Weekly: we are the champions
7 11/06/2020 Experience Weekly: who shot Erik Fisher?
8 05/06/2020 Experience Weekly: firstly, sport, and then wine
9 29/05/2020 Experience Weekly: 600k from the charity auction 
10 22/05/2020 Experience Weekly: nostalgia about sushi (on Wednesdays) po sredam 
11 15/05/2020 Experience Weekly: where I was born
12 30/04/2020 Experience Weekly: Who ya gonna call? Kuzbassters!
13 24/04/2020 Experience Weekly: convincing with a knife
14 17/04/2020 Experience Weekly: winners, winners – chicken dinners 
15 10/04/2020 Experience Weekly: CEO wi

Выберем одно сообщение и посмотрим, какую информацию о нем можно получить

In [10]:
mail=mails[68]

Кому отправленно сообщение

In [12]:
print(mail.TO)
print(mail.CC)
print(mail.BCC)

MSW-Firm Members; KAZ-Firm Members




Кто отправил: почтовый ящик и реальный отправитель

In [13]:
print(str(mail.Sender))
print(str(mail.SendUsingAccount))

CI Experience
Denis_Bernikov@mckinsey.com


Тема письма и текст сообщения. Не пугайтесь, что текст сообщения такой страшный, длинный и непонятный. Это язык HTML, который содержит в себе не только информацию по тексту сообщения, но и по всему форматингу: выравнивания шрифт, ссылки, картинки и многое другое - то есть все, чтобы сообщение выглядело именно так, как оно выглядит.

In [14]:
print(mail.Subject)
print(mail.HTMLBody)

Experience Weekly: secrets of Sports Kings
<html xmlns:v="urn:schemas-microsoft-com:vml" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:w="urn:schemas-microsoft-com:office:word" xmlns:x="urn:schemas-microsoft-com:office:excel" xmlns:m="http://schemas.microsoft.com/office/2004/12/omml" xmlns="http://www.w3.org/TR/REC-html40"><head><meta name=Generator content="Microsoft Word 15 (filtered medium)"><!--[if !mso]><style>v\:* {behavior:url(#default#VML);}
o\:* {behavior:url(#default#VML);}
w\:* {behavior:url(#default#VML);}
.shape {behavior:url(#default#VML);}
</style><![endif]--><style><!--
/* Font Definitions */
@font-face
	{font-family:"Cambria Math";
	panose-1:2 4 5 3 5 4 6 3 2 4;}
@font-face
	{font-family:Calibri;
	panose-1:2 15 5 2 2 2 4 3 2 4;}
@font-face
	{font-family:Georgia;
	panose-1:2 4 5 2 5 4 5 2 3 3;}
@font-face
	{font-family:"Arial \(Body\)";}
/* Style Definitions */
p.MsoNormal, li.MsoNormal, div.MsoNormal
	{margin:0cm;
	margin-bottom:.0001pt;
	font-size:11.0pt;
	f

Но есть возможность показывать только текст.

Практически только текст: будут показываться еще ссылки и отступы, но, все равно, заведомо понятнее и короче :)

In [15]:
mail.Body

' \r\n\r\n\r\n\r\n \r\n\r\nFizkult-privet!\r\n\r\n \r\n\r\n \r\n\r\n \r\n\r\n \r\n\r\n \r\n\r\nNo, we haven’t confused this newsletter with the one about sports challenges, because this time we’re featuring our 8 winners (Anya Buketova, Katya Udod, Kostya Semishchenko, Natasha Sukharenko, Nikita Dolzhikov, Yulia Vodopyanova, Rustem Karimov, and Dasha Romanova), who share some tips & tricks on how to make sports your 2nd nature (or at least a habit). Even though the summer is over, and proximity of Oktoberfest might call for different kinds of “challenges” these days... So, here are the sports tips from the true champions:\r\n\r\n \r\n\r\n \r\n\r\n \r\n\r\n \r\n\r\n\r\n\r\n \r\n\r\n \r\n\r\n \r\n\r\n\r\n\r\nIt’s better to start slow but today (than fast but never). Don’t immediately push yourself to the limit. It’s ok to run just 1 km and gradually increase your load as you feel comfortable. However, it’s important to not postpone and start today. So, if you’re heading to the bar right 

Это только основная информация о письме, что можно получить. Но вообще с помощью Python вы сможете вычленить всю информацию о письме, которая доступна. Для больших деталей - смотрите на раздел Properties в [документации](https://docs.microsoft.com/en-us/office/vba/api/outlook.mailitem).

## Создание и отправка писем

С помощью Python можно не только находить письма и извлекать из них полезную информацию, но можно и создавать письма.

Чтобы это сделать для начала надо создать объект-письмо.

In [29]:
olMailItem = 0x0
obj = win32com.client.Dispatch("Outlook.Application")
newMail = obj.CreateItem(olMailItem)

Мы создали объект-письмо `newMail` и можем даже на него посмотреть с помощью функции `Display`

In [17]:
newMail.Display()

В этот момент у меня открылось пустое письмо в Outlook, давайте его чем-нибудь наполним

In [18]:
newMail.Subject='Тестовое письмо для лекции'
newMail.HTMLBody='''Привет!<br><br>Какой-то очень важный текст<br>Денис'''
newMail.Display()

`<br>` — это тэг HTML для начала новой строки. Также в открывшемся письме мы заметим, что текст написан шрифтом Times New Roman 12 размера, поэтому давайте поставим привычный Calibri 11 размера

In [30]:
text='''Привет!<br><br>Какой-то очень важный текст<br>Денис'''
newMail.HTMLBody="<p style='font-size:11.0pt;font-family:Calibri,sans-serif;'>"+text+"</p>"
newMail.Display()

Добавим еще получателей этого письма

In [31]:
newMail.To= 'Denis_Bernikov@mckinsey.com'
newMail.CC= 'Denis_Bernikov@mckinsey.com'
newMail.BCC= 'Denis_Bernikov@mckinsey.com'
newMail.Display()

Теперь мы можем сохранить письмо как драфт

In [21]:
path=r'C:\Users\Denis Bernikov\OneDrive - McKinsey & Company\Desktop' #папка, куда сохраняется файл
newMail.SaveAs(path+'\\'+'Test Mail.oft', 2) #2 здесь значит формат oft

Либо можем сразу отправить сообщение

In [22]:
newMail.Send()

Также можем отправить сообщение не со своего основного аккаунта

In [32]:
newMail.SentOnBehalfOfName = 'ci_experience@mckinsey.com'
newMail.Send()

Не обязательно создавать новое сообщение, Python позволяет отвечать и форвардить письма

In [33]:
replay_message = mail.Reply() 
original_text=replay_message.HTMLBody #Это нужно, чтобы в отправленном письме была история переписки
replay_message.HTMLBody='some new text'+original_text
replay_message.Subject='New subject'
replay_message.Display()

С помощью Python'а вы сможете сэкономить много времени, когда вам надо взаимодействовать с большим количеством однотипных писем, при этом снизится вероятность ошибки. Разумеется, функционал заведомо шире, чем описан в этой лекции, но здесь собраны основы, которые могут вам пригодиться для оптимизации.

Чтобы изучить дополнительные функции —  изучите [документацию VBA к Outlook](https://docs.microsoft.com/en-us/office/vba/api/outlook.mailitem) (на ее основе сделана библиотека win32)